In [5]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from tqdm import tqdm

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Prepare dataset
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_train = x_train[:10000] # use only 10,000 images for faster training
y_train = y_train[:10000]
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=0)

# Define quantum circuit
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def circuit(params, x):
    # Encode image as quantum state
    for i in range(len(x)):
        qml.RY(x[i]*params[i], wires=i)
    # Apply Hadamard gate to all wires
    for i in range(len(x)):
        qml.Hadamard(wires=i)
    # Perform SWAP test between two quantum states
    qml.Hadamard(wires=3)
    qml.CSWAP(wires=[3,1,2])
    qml.Hadamard(wires=3)
    return qml.expval(qml.PauliZ(3))

# Define contrastive loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    loss = y_true * tf.square(tf.maximum(0., margin - y_pred)) + (1 - y_true) * tf.square(y_pred)
    return tf.reduce_mean(loss)

# Define model architecture
inputs1 = keras.Input(shape=(28, 28))
inputs2 = keras.Input(shape=(28, 28))
flatten = layers.Flatten()
dense = layers.Dense(128, activation="relu")
outputs = layers.Lambda(lambda x: circuit(x[0], x[1]))([dense(flatten(inputs1)), dense(flatten(inputs2))])
model = keras.Model(inputs=[inputs1, inputs2], outputs=outputs)

# Compile model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=contrastive_loss)

# Train model
batch_size = 64
epochs = 10
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    for i in tqdm(range(0, len(x_train), batch_size)):
        x_batch_1 = x_train[i:i+batch_size]
        x_batch_2 = np.copy(x_batch_1)
        y_batch = np.where(y_train[i:i+batch_size] == y_train[i], 1, 0)
        for j in range(len(x_batch_2)):
            x_batch_2[j] = np.roll(x_batch_2[j], np.random.randint(1, 28), axis=1) # randomly shift pixels
        loss = model.train_on_batch([x_batch_1, x_batch_2], y_batch)
    y_pred = []
    for i in range(0, len(x_val), batch_size):
        x_batch_1 = x_val[i:i+batch_size]
        x_batch_2 = np.copy(x_batch_1)
    for j in range(len(x_batch_2)):
        x_batch_2[j] = np.roll(x_batch_2[j], np.random.randint(1, 28), axis=1) # randomly shift pixels
    y_pred_batch = model.predict([x_batch_1, x_batch_2])
    y_pred_batch = np.where(y_pred_batch > 0.5, 1, 0)
    y_pred.extend(y_pred_batch)
acc = accuracy_score(y_val, y_pred)
print("Validation accuracy: {:.2f}%".format(acc * 100))



TypeError: Exception encountered when calling layer "lambda_1" (type Lambda).

len is not well defined for a symbolic Tensor (Placeholder_1:0). Please call `x.shape` rather than `len(x)` for shape information.

Call arguments received by layer "lambda_1" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)']
  • mask=None
  • training=None

In [3]:
pip install qiskit

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device('default.qubit', wires=2*28*28)

@qml.qnode(dev)
def quantum_state(x1, x2, params):
    x = np.concatenate([x1, x2])
    for i in range(len(x)):
        qml.RY(x[i]*params[i], wires=i)
    return qml.state()


ValueError: Maximum allowed dimension exceeded

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity

dev = qml.device('default.qubit', wires=2*28*28+1)

@qml.qnode(dev)
def quantum_circuit(x1, x2, params):
    x = np.concatenate([x1, x2])
    for i in range(len(x)):
        qml.RY(x[i]*params[i], wires=i)

    qml.Hadamard(wires=2*28*28)
    for i in range(2*28*28):
        qml.CSWAP(wires=[i, 2*28*28, i+28*28])
    qml.Hadamard(wires=2*28*28)
    
    return qml.expval(qml.PauliZ(wires=2*28*28))

def fidelity(x1, x2, params):
    overlap = quantum_circuit(x1, x2, params)
    return (overlap + 1) / 2

def contrastive_loss(x1_batch, x2_batch, y_batch, params, margin=1):
    similarity = cosine_similarity(x1_batch, x2_batch)
    similarity = (similarity + 1) / 2
    
    y = tf.cast(y_batch, dtype=tf.float32)
    loss_same = y * (1 - similarity)**2
    loss_diff = (1 - y) * tf.maximum(0, similarity - margin)**2
    
    loss = tf.reduce_mean(loss_same + loss_diff)
    return loss

def train(model, x_train, y_train, batch_size=32, epochs=10, learning_rate=0.1):
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    num_batches = len(x_train) // batch_size
    
    for epoch in range(epochs):
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = start_idx + batch_size
            x1_batch = x_train[start_idx:end_idx]
            x2_batch = x_train[start_idx:end_idx]
            y_batch = (y_train[start_idx:end_idx] == y_train.reshape(-1, 1)[start_idx:end_idx]).astype(int)
            
            with tf.GradientTape() as tape:
                params = model.trainable_variables
                loss = contrastive_loss(x1_batch, x2_batch, y_batch, params)
                
            grads = tape.gradient(loss, params)
            opt.apply_gradients(zip(grads, params))
            
        y_pred = np.array([fidelity(x1, x2, model.trainable_variables) for x1, x2 in zip(x_train, x_train)])
        y_pred[y_pred < 0.5] = 0
        y_pred[y_pred >= 0.5] = 1
        acc = np.mean(y_pred == y_train)
        
        print("Epoch {} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, loss, acc))
